## EQUAL-WEIGHT S&P 500 INDEX FUND
S&P 500 is the world's most popular stock market index. THe largest fund that is benchmarked to this index is the SPDR S&P 500 ETF Trust.

### Objective
Create a python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

### Equal-weight
A type of weighting that gives same weight, or importance, to each stock in a portfolio or index fund
e.g smallest companies are given equal weight to largest companies in an equal-weight index fund or portfolio, regardless of size of company or sector.

In [1]:
!pip install numpy
!pip install pandas
!pip install xlsxwriter


You should consider upgrading via the '/Users/anaistang/.virtualenvs/web/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/anaistang/.virtualenvs/web/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/anaistang/.virtualenvs/web/bin/python -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math 

## Importing list of stocks 
Importing constituents of the S&P 500

Constituents change over time, so in an ideal world you would connect directly to the index provider e.g (Standard and Poor's) and pull their real-time constituents on a regular basis.


In [3]:
stocks = pd.read_csv('starter/sp_500_stocks.csv')
stocks.head(11)

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
5,ABC
6,ABMD
7,ABT
8,ACN
9,ADBE


## Acquiring API token from IEX Cloud API token 
https://cloud.iexapis.com/




In [4]:
from starter.secrets import IEX_CLOUD_API_TOKEN
print(IEX_CLOUD_API_TOKEN)

Tpk_c2ab9004d60340108c152c28159204f0


## Making API call to IEX Cloud 
- Market capitalisation for each stock 
- Price of each stock 

In [5]:
symbol = 'IBM'
# Real-time stock prices
base_url = 'https://sandbox.iexapis.com'
query_params = f'?token={IEX_CLOUD_API_TOKEN}'
api_call = f"{base_url}/stable/stock/{symbol}/quote{query_params}"
print(api_call)
data = requests.get(api_call)
response = data.json()

https://sandbox.iexapis.com/stable/stock/IBM/quote?token=Tpk_c2ab9004d60340108c152c28159204f0


In [6]:
price = response['latestPrice']
market_cap = response['marketCap']
print(price, market_cap/100000000)

151.56 1322.60014574


## Adding stocks data to a pandas data frame (tabular data)

### Pandas Series
 - One dimensional labelled array 
 - Axis labels are called index 
 - Pandas series is a column in an excel sheet
 - Can only contain single list with index
 
 ### Dataframe
 - Can be made of more than one series
 - Dataframe is collection of series that can be used to analyse data


In [7]:
columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
df = pd.DataFrame(columns=columns)
# every row, column is a pandas series 
# initialising dataframe and append a pandas series (a list, numpy array, dict can be turned into pandas series)
df.append(
    pd.Series(
        [
    symbol, 
    price, 
    market_cap, 
    'N/A'
        ],
    index=columns
    ),
    ignore_index=True
)


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,IBM,151.56,132260014574,N/A


In [8]:
# looping through tickers in list of stocks 
df = pd.DataFrame(columns=columns)
for stock in stocks['Ticker'][:5]:
    base_url = 'https://sandbox.iexapis.com'
    query_params = f'?token={IEX_CLOUD_API_TOKEN}'
    api_call = f"{base_url}/stable/stock/{stock}/quote{query_params}"
    # slow, next iteration do batch requests
    data = requests.get(api_call).json()
    df = df.append(
        pd.Series(
            [
            stock, 
            data['latestPrice'],
            data['marketCap'],
            'N/A'
            ],
             index=columns,
        ),
        ignore_index=True,
    )
df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,138.50,42419306696,N/A
1,AAL,25.00,15619281639,N/A
2,AAP,200.95,13015412640,N/A
3,AAPL,131.83,2123294755521,N/A
4,ABBV,115.86,199538127027,N/A


## Using Batch API Calls to Improve Performance
- Easiest ways to improve performance of code
- HTTP requests are typically slowest components of a script 
- API providers will often give discounted rates for using batch API calls since they are easier for API provider to respond to 
- IEX cloud limits batch API calls to 100 tickers per request


In [9]:
# generator yields one item at a time, generates only when in demand 
# list comp, python reserves memory for whole list, list comp will create entire list in memory first
# generators don't support indexing or slicing and cannot be added to lists
# generators should be used if iterating once, memory efficient
# if you want to store and use generated results, better off with list comp, list comp is fater
def chunks(lst, n):
    """Yield successive n-sized chunks from list"""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        

In [10]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for index, symbol in enumerate(symbol_groups):
    symbol_strings.append(','.join(symbol))
    
df = pd.DataFrame(columns=columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        df = df.append(
        pd.Series(
        [
            symbol, 
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['marketCap'],
            'N/A'
        ],
        index=columns
           # Do not use index values on concatenation axis
        ),ignore_index=True)
df
    

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,143.60,42833813145,N/A
1,AAL,24.90,15808043611,N/A
2,AAP,194.84,12620956274,N/A
3,AAPL,126.72,2195348059156,N/A
4,ABBV,112.97,203534948001,N/A
...,...,...,...,...
500,YUM,124.58,36322357554,N/A
501,ZBH,166.29,34414076784,N/A
502,ZBRA,526.14,28769487836,N/A
503,ZION,59.99,9985665620,N/A


In [11]:
portfolio_size = input('Enter value of portfolio:\n')
try:
    val = float(portfolio_size)
except ValueError:
    print('Please enter an integer')
    portfolio_size = input('Enter value of portfolio:\n')
    val = float(portfolio_size)

Enter value of portfolio:
10000000


In [18]:
position_size = val/len(df.index)
number_of_apple_shares = position_size/500
math.floor(number_of_apple_shares)
for i in range(0, len(df.index)):
    


39